In [69]:
import pandas as pd
import numpy as np
import math
import sys
import os
# /Users/liguangyu/Documents/GitHub/gps-spoofing-detection/data/drive-me-not/trace4.csv



In [70]:
ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
print(ROOT_DIR)

# Add the project root directory to the Python path
sys.path.append(ROOT_DIR)
print(sys.path)
from utils.haversine import haversine
from utils.generate_spoof import average_speed,generate_spoof_trace
from utils.estimate_path_err import estimate_path_err

/Users/liguangyu/Documents/GitHub/gps-spoofing-detection
['/Users/liguangyu/Documents/GitHub/gps-spoofing-detection/notebooks', '/Users/liguangyu/miniforge3/envs/tf26/lib/python310.zip', '/Users/liguangyu/miniforge3/envs/tf26/lib/python3.10', '/Users/liguangyu/miniforge3/envs/tf26/lib/python3.10/lib-dynload', '', '/Users/liguangyu/miniforge3/envs/tf26/lib/python3.10/site-packages', '/Users/liguangyu/miniforge3/envs/tf26/lib/python3.10/site-packages/transformers-4.39.0.dev0-py3.10.egg', '/Users/liguangyu/miniforge3/envs/tf26/lib/python3.10/site-packages/safetensors-0.4.2-py3.10-macosx-11.1-arm64.egg', '/Users/liguangyu/miniforge3/envs/tf26/lib/python3.10/site-packages/tokenizers-0.15.2-py3.10-macosx-11.1-arm64.egg', '/Users/liguangyu/miniforge3/envs/tf26/lib/python3.10/site-packages/huggingface_hub-0.21.4-py3.10.egg', '/Users/liguangyu/miniforge3/envs/tf26/lib/python3.10/site-packages/fsspec-2024.2.0-py3.10.egg', '/Users/liguangyu/Documents/GitHub/gps-spoofing-detection', '/Users/liguan

## add distance columns to every row 

In [71]:

data= pd.read_csv("../data/drive-me-not/trace4.csv")
cell_data=pd.read_csv("../data/drive-me-not/CellsDatabase.csv")


cell_data_lalon=cell_data
cell_data_lalon=cell_data_lalon.rename(columns={'cell':'CID','area':'LAC','net':'MNC'})
data_distance=pd.merge(data,cell_data_lalon,on=['CID','LAC','MNC'],how='left')
data_distance['distance']=data_distance.apply(lambda row: haversine([row['GPS_lat'],row['lat']],[row['GPS_long'],row['lon']]),axis=1)
data_distance.to_csv('../data/drive-me-not/processed/trace4_cell.csv')


## drop repeated location with different time

In [72]:
data_unique = data.drop_duplicates(subset=["GPS_lat", "GPS_long","CID"])
data_unique = data_unique.reset_index(drop=True)
data_unique["spoofed"] = 0
data_unique.to_csv("../data/drive-me-not/processed/trace4_unique.csv")


## generate spoofed trace from middle of dataset

In [73]:
data_spoofed = data_unique.drop_duplicates(subset=["GPS_lat", "GPS_long"])
data_spoofed = data_spoofed.reset_index(drop=True)
start = int(0.5 * data_spoofed.shape[0])
print(start)
total = data_spoofed.iloc[-1]["Time"] - data_spoofed.iloc[0]["Time"]
remain_time = data_spoofed.iloc[-1]["Time"] - data_spoofed.iloc[start]["Time"]
speed = average_speed(data_spoofed, start)
# random pick a direction
angle = np.random.uniform(0, 2 * np.pi)
data_spoofed = generate_spoof_trace(data_spoofed, speed, angle, start)
data_spoofed.to_csv("../data/drive-me-not/processed/trace4_spoofed.csv")

233


In [74]:
data_unique.columns

Index(['GPS_lat', 'GPS_long', 'Network_lat', 'Network_long', 'Time',
       'Anchor_Number', 'Type', 'Registered', 'CID', 'LAC', 'MCC', 'MNC',
       'dBm', 'level', 'spoofed'],
      dtype='object')

## merge spoofed data with the original unique data

In [75]:
# origin_data = pd.read_csv(
#     "/Users/liguangyu/Downloads/gps-spoofing-detection-cellular-master/trace4_unique.csv"
# )
# spoofed_data = pd.read_csv(
#     "/Users/liguangyu/Downloads/gps-spoofing-detection-cellular-master/trace4_spoofed.csv"

# )
import copy
merged=pd.merge(data_unique,data_spoofed,on=["Time"],how='left')
merged.to_csv('merged.csv')
# origin_data[["GPS_lat", "GPS_long"]]=merged[["GPS_lat", "GPS_long"]].combine_first(origin_data[["GPS_lat", "GPS_long"]])
origin_data=copy.deepcopy(data_unique)
origin_data[["GPS_lat", "GPS_long"]]=merged[["GPS_lat_y", "GPS_long_y"]]
origin_data['spoofed']=merged['spoofed_y']
origin_data=origin_data.dropna()
origin_data = origin_data.reset_index(drop=True)
mdata=pd.merge(origin_data,cell_data_lalon,on=['CID','LAC','MNC'],how='left')
origin_data['lat']=mdata['lat']
origin_data['lon']=mdata['lon']
origin_data=origin_data.dropna()
origin_data.to_csv('../data/drive-me-not/processed/spoofed_trace4_unique_cell.csv')
# origin_data.to_csv('full_spoofed_trace4.csv')

In [76]:
merged.columns

Index(['GPS_lat_x', 'GPS_long_x', 'Network_lat_x', 'Network_long_x', 'Time',
       'Anchor_Number_x', 'Type_x', 'Registered_x', 'CID_x', 'LAC_x', 'MCC_x',
       'MNC_x', 'dBm_x', 'level_x', 'spoofed_x', 'GPS_lat_y', 'GPS_long_y',
       'Network_lat_y', 'Network_long_y', 'Anchor_Number_y', 'Type_y',
       'Registered_y', 'CID_y', 'LAC_y', 'MCC_y', 'MNC_y', 'dBm_y', 'level_y',
       'spoofed_y'],
      dtype='object')

## test the max difference

In [77]:
origin_data['e_lat']=0.0
origin_data['e_lon']=0.0
origin_data['difference']=0.0
origin_data = estimate_path_err(origin_data, expF=20, qthr=0.9)
origin_data.to_csv('../data/drive-me-not/processed/estimated_spoofed_trace4.csv')
print(np.argmax(origin_data['difference'].values))
# data_spoofed = estimate_path_err(data_spoofed, expF=20, qthr=0.9)
# print(np.argmax(data_spoofed['difference'].values))

Estimating path error for 20... 
0        True
1        True
2        True
3        True
4        True
        ...  
5859    False
5860    False
5861    False
5862    False
5863    False
Name: Time, Length: 5864, dtype: bool
0       False
1       False
2       False
3       False
4       False
        ...  
5859    False
5860    False
5861    False
5862    False
5863    False
Name: Time, Length: 5864, dtype: bool
0       False
1       False
2       False
3       False
4       False
        ...  
5859    False
5860    False
5861    False
5862    False
5863    False
Name: Time, Length: 5864, dtype: bool
0       False
1       False
2       False
3       False
4       False
        ...  
5859    False
5860    False
5861    False
5862    False
5863    False
Name: Time, Length: 5864, dtype: bool
0       False
1       False
2       False
3       False
4       False
        ...  
5859    False
5860    False
5861    False
5862    False
5863    False
Name: Time, Length: 5864, dtype: bool
0      